In [5]:
import random

import utils.utils as utils
import utils.datasets as datasets
import utils.py_evm as py_evm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms, models
import dlib
import cv2
from tqdm import tqdm
from PIL import Image

pd.set_option('display.max_columns', 50)
%load_ext autoreload
%autoreload 2

There is a bug in the original code
Prepare.py line 341. If the image is float it's max should be 1, not 255. This causes the HSV image to have values of -10000

In [6]:
#make sure everything is deterministic
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
device = torch.device("cuda:0")

In [8]:
df, load_data = datasets.megc(color=True)

In [7]:
df, load_data = datasets.megc("raw", color=True)

/home/tvaranka/.local/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
n_samples = df.shape[0]

In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../shape_predictor_68_face_landmarks.dat")

In [43]:
def mner_preprocess(onset, apex):
    onset = (onset * 255.0).astype("uint8")
    apex = (apex * 255.0).astype("uint8")
    onset_det = detector(onset, 1)[0] if detector(onset, 1) else detector(onset, 0)[0]
    apex_dets = detector(apex, 1)
    if len(apex_dets) > 0:
        apex_det = apex_dets[0]
    else:
        apex_det = onset_det
    #apex_det = detector(apex, 1)[0]

    onset_face = dlib.full_object_detections()
    apex_face = dlib.full_object_detections()

    onset_face.append(predictor(onset, onset_det))
    apex_face.append(predictor(apex, apex_det))

    onset_crops = dlib.get_face_chips(onset, onset_face, size=320)
    apex_crops = dlib.get_face_chips(apex, apex_face, size=320)

    #Not used, but are in the original code
    #onset_crop = onset_crops[0][:280, 50:270]
    #apex_crop = apex_crops[0][:280, 50:270]

    onset_g = cv2.cvtColor(onset_crops[0], cv2.COLOR_RGB2GRAY)
    apex_g = cv2.cvtColor(apex_crops[0], cv2.COLOR_RGB2GRAY)

    pic_size = onset_crops[0].shape
    hsv = np.zeros(pic_size)
    hsv[:, :, 1] = cv2.cvtColor(apex_crops[0], cv2.COLOR_RGB2HSV)[:, :, 1]

    flow = cv2.calcOpticalFlowFarneback(onset_g, apex_g, flow=None,
                                        pyr_scale=0.5, levels=1, winsize=15,
                                        iterations=2,
                                        poly_n=5, poly_sigma=1.1, flags=0)

    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[:, :, 0] = ang * (180 / np.pi / 2)
    hsv[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    hsv = np.asarray(hsv, dtype=np.float32)
    # This line is added to avoid the bug
    hsv /= 255.0
    rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return rgb_flow


In [49]:
flows = np.zeros((n_samples, 3, 320, 320))
for i, video in enumerate(tqdm(load_data, total=n_samples)):
    mm_video = py_evm.magnify(video)
    flow = mner_preprocess(mm_video[0], mm_video[df.loc[i, "apexf"]])
    flows[i] = flow.transpose(2, 0, 1)



  0%|          | 0/442 [00:00<?, ?it/s]

  0%|          | 1/442 [00:01<09:57,  1.35s/it]

  0%|          | 2/442 [00:02<08:59,  1.23s/it]

  1%|          | 3/442 [00:03<07:59,  1.09s/it]

  1%|          | 4/442 [00:03<07:25,  1.02s/it]

  1%|          | 5/442 [00:04<07:23,  1.01s/it]

  1%|▏         | 6/442 [00:05<07:25,  1.02s/it]

  2%|▏         | 7/442 [00:06<07:03,  1.03it/s]

  2%|▏         | 8/442 [00:08<07:44,  1.07s/it]

  2%|▏         | 9/442 [00:09<09:02,  1.25s/it]

  2%|▏         | 10/442 [00:11<10:26,  1.45s/it]

  2%|▏         | 11/442 [00:13<11:27,  1.60s/it]

  3%|▎         | 12/442 [00:15<11:36,  1.62s/it]

  3%|▎         | 13/442 [00:16<10:45,  1.50s/it]

  3%|▎         | 14/442 [00:18<11:38,  1.63s/it]

  3%|▎         | 15/442 [00:19<10:11,  1.43s/it]

  4%|▎         | 16/442 [00:21<10:53,  1.53s/it]

  4%|▍         | 17/442 [00:22<10:05,  1.42s/it]

  4%|▍         | 18/442 [00:23<09:54,  1.40s/it]

  4%|▍         | 19/442 [00:25<10:10,  1.44s/it]

  5%|▍         |

 33%|███▎      | 145/442 [03:17<07:15,  1.47s/it]

 33%|███▎      | 146/442 [03:18<06:17,  1.28s/it]

 33%|███▎      | 147/442 [03:19<06:37,  1.35s/it]

 33%|███▎      | 148/442 [03:21<06:53,  1.41s/it]

 34%|███▎      | 149/442 [03:22<06:49,  1.40s/it]

 34%|███▍      | 150/442 [03:23<06:40,  1.37s/it]

 34%|███▍      | 151/442 [03:25<07:03,  1.46s/it]

 34%|███▍      | 152/442 [03:27<07:50,  1.62s/it]

 35%|███▍      | 153/442 [03:29<08:36,  1.79s/it]

 35%|███▍      | 154/442 [03:30<07:28,  1.56s/it]

 35%|███▌      | 155/442 [03:31<07:01,  1.47s/it]

 35%|███▌      | 156/442 [03:33<07:25,  1.56s/it]

 36%|███▌      | 157/442 [03:35<07:22,  1.55s/it]

 36%|███▌      | 158/442 [03:36<06:50,  1.45s/it]

 36%|███▌      | 159/442 [03:37<06:58,  1.48s/it]

 36%|███▌      | 160/442 [03:39<07:27,  1.59s/it]

 36%|███▋      | 161/442 [03:41<07:53,  1.69s/it]

 37%|███▋      | 162/442 [03:42<07:15,  1.56s/it]

 37%|███▋      | 163/442 [03:44<07:44,  1.67s/it]

 37%|███▋      | 164/442 [03:46

 65%|██████▌   | 288/442 [21:22<23:08,  9.01s/it]

 65%|██████▌   | 289/442 [21:30<22:12,  8.71s/it]

 66%|██████▌   | 290/442 [21:37<20:46,  8.20s/it]

 66%|██████▌   | 291/442 [21:48<22:39,  9.01s/it]

 66%|██████▌   | 292/442 [21:52<18:57,  7.59s/it]

 66%|██████▋   | 293/442 [21:56<16:02,  6.46s/it]

 67%|██████▋   | 294/442 [22:01<14:27,  5.86s/it]

 67%|██████▋   | 295/442 [22:10<16:51,  6.88s/it]

 67%|██████▋   | 296/442 [22:23<21:09,  8.70s/it]

 67%|██████▋   | 297/442 [22:30<19:37,  8.12s/it]

 67%|██████▋   | 298/442 [22:36<17:54,  7.46s/it]

 68%|██████▊   | 299/442 [22:47<20:25,  8.57s/it]

 68%|██████▊   | 300/442 [22:55<19:47,  8.36s/it]

 68%|██████▊   | 301/442 [23:02<18:44,  7.98s/it]

 68%|██████▊   | 302/442 [23:13<20:47,  8.91s/it]

 69%|██████▊   | 303/442 [23:24<21:56,  9.47s/it]

 69%|██████▉   | 304/442 [23:30<19:46,  8.60s/it]

 69%|██████▉   | 305/442 [23:38<18:55,  8.29s/it]

 69%|██████▉   | 306/442 [23:49<20:38,  9.11s/it]

 69%|██████▉   | 307/442 [23:57

 97%|█████████▋| 430/442 [1:10:12<05:04, 25.42s/it]

 98%|█████████▊| 431/442 [1:10:33<04:25, 24.15s/it]

 98%|█████████▊| 432/442 [1:10:55<03:57, 23.71s/it]

 98%|█████████▊| 433/442 [1:11:07<03:01, 20.13s/it]

 98%|█████████▊| 434/442 [1:11:17<02:15, 16.90s/it]

 98%|█████████▊| 435/442 [1:11:34<01:59, 17.09s/it]

 99%|█████████▊| 436/442 [1:11:52<01:43, 17.26s/it]

 99%|█████████▉| 437/442 [1:12:16<01:36, 19.30s/it]

 99%|█████████▉| 438/442 [1:12:31<01:12, 18.16s/it]

 99%|█████████▉| 439/442 [1:12:43<00:49, 16.34s/it]

100%|█████████▉| 440/442 [1:13:09<00:38, 19.00s/it]

100%|█████████▉| 441/442 [1:13:41<00:23, 23.14s/it]

100%|██████████| 442/442 [1:14:12<00:00, 10.07s/it]


In [12]:
le = LabelEncoder()
labels = le.fit_transform(df["emotion"])
dataset = le.fit_transform(df["dataset"])

In [13]:
class MEData(Dataset):
    def __init__(self, frames, labels, transform=None):
        self.frames = frames
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return self.frames.shape[0]
    
    def __getitem__(self, idx):
        sample = self.frames[idx]
        if self.transform:
            sample = Image.fromarray((sample.transpose(1, 2, 0) * 255).astype("uint8"))
            sample = self.transform(sample)
        label = self.labels[idx]
        return {"image": sample, "label": label}

In [14]:
class Resnet18_pt_mcn_removed(nn.Module):
    def __init__(self):
        super(Resnet18_pt_mcn_removed, self).__init__()
        self.meta = {'mean': [0.485, 0.456, 0.406],
                     'std': [0.229, 0.224, 0.225],
                     'imageSize': [224, 224]}
        self.backbone = models.resnet18(pretrained=True)
        self.backbone = torch.nn.Sequential(*(list(self.backbone.children())[:-2]))
        self.features_8 = nn.AvgPool2d(kernel_size=[7, 7], stride=[1, 1], padding=0, ceil_mode=False, count_include_pad=False)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
    def forward(self, x):
        features_7 = self.backbone(x)
        
        top = features_7[:, :, :3]
        bottom = features_7[:, :, 3:]
        
        top = self.avgpool(top)
        bottom = self.avgpool(bottom)
        
        top = top.view(top.size(0), -1)
        bottom = bottom.view(bottom.size(0), -1)
        
        return top, bottom
    
    
class FC_3(nn.Module):
    def __init__(self, num_classes=3):
        super(FC_3, self).__init__()
        self.fc_3 = nn.Sequential(
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Dropout(0.1)
        )

    def forward(self, x):
        x = self.fc_3(x)

        return x

    
class Classifier(nn.Module):
    def __init__(self, input_classes=3, num_classes=3):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(64 * input_classes, num_classes)

    def forward(self, x):
        x = self.fc(x)

        return x

In [15]:
def Flow_Part_npic(num_pic=2, num_classes=3, **kwargs):

    model = {}

    model['resnet'] = Resnet18_pt_mcn_removed()
    model['fc_top'] = FC_3()
    model['fc_bottom'] = FC_3()
    model['classifier'] = Classifier(input_classes=num_pic, num_classes=num_classes)
    model['classifier_top'] = Classifier(input_classes=1, num_classes=num_classes)
    model['classifier_bottom'] = Classifier(input_classes=1, num_classes=num_classes)

    return model

In [16]:
transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomCrop((224, 224), pad_if_needed=True),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])

def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0.0)

sssnet
Total f1: 0.7617842853527486, SMIC: 0.7241661846925004, CASME2: 0.8568498168498168, SAMM: 0.6528043905093086

In [17]:
LEARNING_RATE =  0.001

In [25]:
LOSO(flows, df, epochs=100, lr=0.01, weight_decay=0.000001,
     dropout=0.5, batch_size=32)

Train:	Epoch:[0][1/14]   	Acc: 0.312	Label_Loss: 1.3918	Top_Loss: 1.1304	Bottom_Loss: 1.4040	Loss: 3.9263	
Train:	Epoch:[0][8/14]   	Acc: 0.344	Label_Loss: 1.2801	Top_Loss: 1.2615	Bottom_Loss: 1.3924	Loss: 3.9340	
Subject: 006, n=11 | test_f1: 0.2807 |best_f1: 0.2807
Train:	Epoch:[1][1/14]   	Acc: 0.500	Label_Loss: 1.0769	Top_Loss: 1.1487	Bottom_Loss: 1.0673	Loss: 3.2929	
Train:	Epoch:[1][8/14]   	Acc: 0.219	Label_Loss: 1.5872	Top_Loss: 1.5969	Bottom_Loss: 1.5141	Loss: 4.6982	
Subject: 006, n=11 | test_f1: 0.25926 |best_f1: 0.2807
Train:	Epoch:[2][1/14]   	Acc: 0.531	Label_Loss: 0.9751	Top_Loss: 1.0569	Bottom_Loss: 1.1075	Loss: 3.1395	
Train:	Epoch:[2][8/14]   	Acc: 0.562	Label_Loss: 0.8990	Top_Loss: 0.8245	Bottom_Loss: 0.9107	Loss: 2.6342	
Subject: 006, n=11 | test_f1: 0.45 |best_f1: 0.45
Train:	Epoch:[3][1/14]   	Acc: 0.531	Label_Loss: 0.9144	Top_Loss: 0.7261	Bottom_Loss: 1.0349	Loss: 2.6753	
Train:	Epoch:[3][8/14]   	Acc: 0.531	Label_Loss: 1.0093	Top_Loss: 0.8429	Bottom_Loss: 1.0732

Subject: 006, n=11 | test_f1: 0.23529 |best_f1: 0.80702
Train:	Epoch:[31][1/14]   	Acc: 0.938	Label_Loss: 0.2778	Top_Loss: 0.4302	Bottom_Loss: 0.4581	Loss: 1.1661	
Train:	Epoch:[31][8/14]   	Acc: 0.906	Label_Loss: 0.2297	Top_Loss: 0.2833	Bottom_Loss: 0.4421	Loss: 0.9552	
Subject: 006, n=11 | test_f1: 0.42105 |best_f1: 0.80702
Train:	Epoch:[32][1/14]   	Acc: 0.906	Label_Loss: 0.3106	Top_Loss: 0.4212	Bottom_Loss: 0.4797	Loss: 1.2116	
Train:	Epoch:[32][8/14]   	Acc: 0.969	Label_Loss: 0.1453	Top_Loss: 0.2348	Bottom_Loss: 0.2873	Loss: 0.6674	
Subject: 006, n=11 | test_f1: 0.35294 |best_f1: 0.80702
Train:	Epoch:[33][1/14]   	Acc: 0.969	Label_Loss: 0.1990	Top_Loss: 0.3552	Bottom_Loss: 0.2709	Loss: 0.8251	
Train:	Epoch:[33][8/14]   	Acc: 0.906	Label_Loss: 0.3264	Top_Loss: 0.4251	Bottom_Loss: 0.5347	Loss: 1.2861	
Subject: 006, n=11 | test_f1: 0.2807 |best_f1: 0.80702
Train:	Epoch:[34][1/14]   	Acc: 0.844	Label_Loss: 0.4011	Top_Loss: 0.5832	Bottom_Loss: 0.3902	Loss: 1.3745	
Train:	Epoch:[34][8/1

Train:	Epoch:[61][8/14]   	Acc: 0.969	Label_Loss: 0.0784	Top_Loss: 0.1507	Bottom_Loss: 0.1336	Loss: 0.3627	
Subject: 006, n=11 | test_f1: 0.80702 |best_f1: 0.80702
Train:	Epoch:[62][1/14]   	Acc: 1.000	Label_Loss: 0.0601	Top_Loss: 0.2049	Bottom_Loss: 0.0840	Loss: 0.3489	
Train:	Epoch:[62][8/14]   	Acc: 0.938	Label_Loss: 0.1622	Top_Loss: 0.2025	Bottom_Loss: 0.2052	Loss: 0.5700	
Subject: 006, n=11 | test_f1: 0.38889 |best_f1: 0.80702
Train:	Epoch:[63][1/14]   	Acc: 1.000	Label_Loss: 0.0321	Top_Loss: 0.1106	Bottom_Loss: 0.1142	Loss: 0.2569	
Train:	Epoch:[63][8/14]   	Acc: 0.938	Label_Loss: 0.1315	Top_Loss: 0.3579	Bottom_Loss: 0.1400	Loss: 0.6295	
Subject: 006, n=11 | test_f1: 0.42105 |best_f1: 0.80702
Train:	Epoch:[64][1/14]   	Acc: 1.000	Label_Loss: 0.0521	Top_Loss: 0.0977	Bottom_Loss: 0.1736	Loss: 0.3234	
Train:	Epoch:[64][8/14]   	Acc: 1.000	Label_Loss: 0.0605	Top_Loss: 0.2321	Bottom_Loss: 0.1235	Loss: 0.4162	
Subject: 006, n=11 | test_f1: 0.2807 |best_f1: 0.80702
Train:	Epoch:[65][1/1

Subject: 006, n=11 | test_f1: 0.2807 |best_f1: 0.80702
Train:	Epoch:[92][1/14]   	Acc: 1.000	Label_Loss: 0.0148	Top_Loss: 0.0516	Bottom_Loss: 0.0355	Loss: 0.1019	
Train:	Epoch:[92][8/14]   	Acc: 0.969	Label_Loss: 0.0331	Top_Loss: 0.0513	Bottom_Loss: 0.1401	Loss: 0.2245	
Subject: 006, n=11 | test_f1: 0.42105 |best_f1: 0.80702
Train:	Epoch:[93][1/14]   	Acc: 1.000	Label_Loss: 0.0041	Top_Loss: 0.0183	Bottom_Loss: 0.0207	Loss: 0.0431	
Train:	Epoch:[93][8/14]   	Acc: 1.000	Label_Loss: 0.0066	Top_Loss: 0.0207	Bottom_Loss: 0.0336	Loss: 0.0609	
Subject: 006, n=11 | test_f1: 0.45 |best_f1: 0.80702
Train:	Epoch:[94][1/14]   	Acc: 1.000	Label_Loss: 0.0099	Top_Loss: 0.0272	Bottom_Loss: 0.0254	Loss: 0.0626	
Train:	Epoch:[94][8/14]   	Acc: 1.000	Label_Loss: 0.0056	Top_Loss: 0.0260	Bottom_Loss: 0.0315	Loss: 0.0631	
Subject: 006, n=11 | test_f1: 0.2807 |best_f1: 0.80702
Train:	Epoch:[95][1/14]   	Acc: 1.000	Label_Loss: 0.0317	Top_Loss: 0.0893	Bottom_Loss: 0.0519	Loss: 0.1729	
Train:	Epoch:[95][8/14]  

Train:	Epoch:[22][1/14]   	Acc: 0.812	Label_Loss: 0.5307	Top_Loss: 0.7197	Bottom_Loss: 0.6064	Loss: 1.8567	
Train:	Epoch:[22][8/14]   	Acc: 0.688	Label_Loss: 0.6500	Top_Loss: 0.6397	Bottom_Loss: 0.6108	Loss: 1.9005	
Subject: 007, n=08 | test_f1: 0.074074 |best_f1: 0.083333
Train:	Epoch:[23][1/14]   	Acc: 0.875	Label_Loss: 0.4472	Top_Loss: 0.5538	Bottom_Loss: 0.5313	Loss: 1.5323	
Train:	Epoch:[23][8/14]   	Acc: 0.875	Label_Loss: 0.3749	Top_Loss: 0.4907	Bottom_Loss: 0.5432	Loss: 1.4088	
Subject: 007, n=08 | test_f1: 0.074074 |best_f1: 0.083333
Train:	Epoch:[24][1/14]   	Acc: 0.656	Label_Loss: 0.7499	Top_Loss: 0.8119	Bottom_Loss: 0.6707	Loss: 2.2325	
Train:	Epoch:[24][8/14]   	Acc: 0.875	Label_Loss: 0.4237	Top_Loss: 0.4394	Bottom_Loss: 0.4810	Loss: 1.3440	
Subject: 007, n=08 | test_f1: 0.074074 |best_f1: 0.083333
Train:	Epoch:[25][1/14]   	Acc: 0.750	Label_Loss: 0.5028	Top_Loss: 0.6390	Bottom_Loss: 0.5579	Loss: 1.6997	
Train:	Epoch:[25][8/14]   	Acc: 0.844	Label_Loss: 0.3873	Top_Loss: 0.6

...

Subject: s8, n=13 | test_f1: 0.16667 |best_f1: 0.44048
Train:	Epoch:[60][1/14]   	Acc: 0.969	Label_Loss: 0.0621	Top_Loss: 0.1965	Bottom_Loss: 0.0584	Loss: 0.3170	
Train:	Epoch:[60][8/14]   	Acc: 0.906	Label_Loss: 0.1786	Top_Loss: 0.1393	Bottom_Loss: 0.2154	Loss: 0.5333	
Subject: s8, n=13 | test_f1: 0.2381 |best_f1: 0.44048
Train:	Epoch:[61][1/14]   	Acc: 1.000	Label_Loss: 0.0725	Top_Loss: 0.1655	Bottom_Loss: 0.0981	Loss: 0.3362	
Train:	Epoch:[61][8/14]   	Acc: 1.000	Label_Loss: 0.0767	Top_Loss: 0.1275	Bottom_Loss: 0.0890	Loss: 0.2932	
Subject: s8, n=13 | test_f1: 0.12963 |best_f1: 0.44048
Train:	Epoch:[62][1/14]   	Acc: 0.969	Label_Loss: 0.1030	Top_Loss: 0.2270	Bottom_Loss: 0.1435	Loss: 0.4735	
Train:	Epoch:[62][8/14]   	Acc: 1.000	Label_Loss: 0.0617	Top_Loss: 0.1760	Bottom_Loss: 0.1121	Loss: 0.3497	
Subject: s8, n=13 | test_f1: 0.28889 |best_f1: 0.44048
Train:	Epoch:[63][1/14]   	Acc: 1.000	Label_Loss: 0.0450	Top_Loss: 0.1100	Bottom_Loss: 0.0545	Loss: 0.2094	
Train:	Epoch:[63][8/14]  

Train:	Epoch:[90][8/14]   	Acc: 1.000	Label_Loss: 0.0057	Top_Loss: 0.0304	Bottom_Loss: 0.0144	Loss: 0.0504	
Subject: s8, n=13 | test_f1: 0.30952 |best_f1: 0.44048
Train:	Epoch:[91][1/14]   	Acc: 0.969	Label_Loss: 0.0458	Top_Loss: 0.0702	Bottom_Loss: 0.0461	Loss: 0.1620	
Train:	Epoch:[91][8/14]   	Acc: 0.938	Label_Loss: 0.0822	Top_Loss: 0.1203	Bottom_Loss: 0.0854	Loss: 0.2879	
Subject: s8, n=13 | test_f1: 0.38889 |best_f1: 0.44048
Train:	Epoch:[92][1/14]   	Acc: 1.000	Label_Loss: 0.0083	Top_Loss: 0.0619	Bottom_Loss: 0.0162	Loss: 0.0864	
Train:	Epoch:[92][8/14]   	Acc: 1.000	Label_Loss: 0.0099	Top_Loss: 0.0364	Bottom_Loss: 0.0219	Loss: 0.0682	
Subject: s8, n=13 | test_f1: 0.25071 |best_f1: 0.44048
Train:	Epoch:[93][1/14]   	Acc: 1.000	Label_Loss: 0.0079	Top_Loss: 0.0245	Bottom_Loss: 0.0148	Loss: 0.0472	
Train:	Epoch:[93][8/14]   	Acc: 1.000	Label_Loss: 0.0070	Top_Loss: 0.0839	Bottom_Loss: 0.0122	Loss: 0.1032	
Subject: s8, n=13 | test_f1: 0.4127 |best_f1: 0.44048
Train:	Epoch:[94][1/14]  

Train:	Epoch:[21][1/14]   	Acc: 0.844	Label_Loss: 0.4007	Top_Loss: 0.6850	Bottom_Loss: 0.4838	Loss: 1.5695	
Train:	Epoch:[21][8/14]   	Acc: 0.625	Label_Loss: 0.7446	Top_Loss: 0.9709	Bottom_Loss: 0.8896	Loss: 2.6051	
Subject: s9, n=04 | test_f1: 0.0 |best_f1: 0.38889
Train:	Epoch:[22][1/14]   	Acc: 0.844	Label_Loss: 0.5123	Top_Loss: 0.5910	Bottom_Loss: 0.5523	Loss: 1.6556	
Train:	Epoch:[22][8/14]   	Acc: 0.812	Label_Loss: 0.4846	Top_Loss: 0.7365	Bottom_Loss: 0.5918	Loss: 1.8129	
Subject: s9, n=04 | test_f1: 0.0 |best_f1: 0.38889
Train:	Epoch:[23][1/14]   	Acc: 0.781	Label_Loss: 0.5798	Top_Loss: 0.6648	Bottom_Loss: 0.6072	Loss: 1.8517	
Train:	Epoch:[23][8/14]   	Acc: 0.906	Label_Loss: 0.2922	Top_Loss: 0.5515	Bottom_Loss: 0.3552	Loss: 1.1989	
Subject: s9, n=04 | test_f1: 0.0 |best_f1: 0.38889
Train:	Epoch:[24][1/14]   	Acc: 0.688	Label_Loss: 0.6245	Top_Loss: 0.6579	Bottom_Loss: 0.5103	Loss: 1.7926	
Train:	Epoch:[24][8/14]   	Acc: 0.875	Label_Loss: 0.2977	Top_Loss: 0.3716	Bottom_Loss: 0.54

Train:	Epoch:[52][1/14]   	Acc: 0.969	Label_Loss: 0.0927	Top_Loss: 0.2854	Bottom_Loss: 0.2325	Loss: 0.6105	
Train:	Epoch:[52][8/14]   	Acc: 0.938	Label_Loss: 0.1095	Top_Loss: 0.2966	Bottom_Loss: 0.1010	Loss: 0.5070	
Subject: s9, n=04 | test_f1: 0.16667 |best_f1: 0.5
Train:	Epoch:[53][1/14]   	Acc: 0.969	Label_Loss: 0.1293	Top_Loss: 0.2398	Bottom_Loss: 0.1347	Loss: 0.5038	
Train:	Epoch:[53][8/14]   	Acc: 0.938	Label_Loss: 0.1857	Top_Loss: 0.2316	Bottom_Loss: 0.2357	Loss: 0.6530	
Subject: s9, n=04 | test_f1: 0.22222 |best_f1: 0.5
Train:	Epoch:[54][1/14]   	Acc: 0.969	Label_Loss: 0.0969	Top_Loss: 0.1552	Bottom_Loss: 0.1585	Loss: 0.4105	
Train:	Epoch:[54][8/14]   	Acc: 1.000	Label_Loss: 0.0599	Top_Loss: 0.1864	Bottom_Loss: 0.1141	Loss: 0.3604	
Subject: s9, n=04 | test_f1: 0.16667 |best_f1: 0.5
Train:	Epoch:[55][1/14]   	Acc: 1.000	Label_Loss: 0.0651	Top_Loss: 0.2442	Bottom_Loss: 0.0793	Loss: 0.3885	
Train:	Epoch:[55][8/14]   	Acc: 1.000	Label_Loss: 0.0884	Top_Loss: 0.2250	Bottom_Loss: 0.12

Subject: s9, n=04 | test_f1: 0.26667 |best_f1: 1.0
Train:	Epoch:[83][1/14]   	Acc: 1.000	Label_Loss: 0.0197	Top_Loss: 0.0999	Bottom_Loss: 0.0415	Loss: 0.1611	
Train:	Epoch:[83][8/14]   	Acc: 1.000	Label_Loss: 0.0138	Top_Loss: 0.0680	Bottom_Loss: 0.0319	Loss: 0.1137	
Subject: s9, n=04 | test_f1: 0.26667 |best_f1: 1.0
Train:	Epoch:[84][1/14]   	Acc: 1.000	Label_Loss: 0.0208	Top_Loss: 0.0488	Bottom_Loss: 0.0243	Loss: 0.0939	
Train:	Epoch:[84][8/14]   	Acc: 1.000	Label_Loss: 0.0133	Top_Loss: 0.0579	Bottom_Loss: 0.0419	Loss: 0.1131	
Subject: s9, n=04 | test_f1: 0.16667 |best_f1: 1.0
Train:	Epoch:[85][1/14]   	Acc: 1.000	Label_Loss: 0.0507	Top_Loss: 0.1018	Bottom_Loss: 0.0898	Loss: 0.2422	
Train:	Epoch:[85][8/14]   	Acc: 1.000	Label_Loss: 0.0190	Top_Loss: 0.0450	Bottom_Loss: 0.0454	Loss: 0.1094	
Subject: s9, n=04 | test_f1: 0.26667 |best_f1: 1.0
Train:	Epoch:[86][1/14]   	Acc: 1.000	Label_Loss: 0.0322	Top_Loss: 0.1690	Bottom_Loss: 0.0623	Loss: 0.2634	
Train:	Epoch:[86][8/14]   	Acc: 1.000	La

EMR with bug and no adversarial loss
Total f1: 0.7618467522303171, SMIC: 0.7267611376522267, CASME2: 0.8386900914664003, SAMM: 0.6452826682770474
EMR without bug and no adversarial loss
Total f1: 0.5098451709853463, SMIC: 0.47025448477263243, CASME2: 0.5572626347082869, SAMM: 0.38079531662740057

In [18]:
def train(train_dataloader, model, criterion, optimizer, epoch, print_freq=7):

    model['resnet'].train()
    model['fc_top'].train()
    model['fc_bottom'].train()
    model['classifier'].train()
    model['classifier_top'].train()
    model['classifier_bottom'].train()

    correct = 0

    for i, sample in enumerate(train_dataloader):
        input, label = sample['image'].to(device), sample['label'].to(device)

        output_resnet_top, output_resnet_bottom = model['resnet'](input)

        output_fc_top = model['fc_top'](output_resnet_top)
        output_fc_bottom = model['fc_bottom'](output_resnet_bottom)

        features = torch.cat((output_fc_top, output_fc_bottom), 1)

        output = model['classifier'](features)
        output_top = model['classifier_top'](output_fc_top)
        output_bottom = model['classifier_bottom'](output_fc_bottom)

        loss_label = criterion(output, label)
        loss_top = criterion(output_top, label)
        loss_bottom = criterion(output_bottom, label)

        loss = loss_label + loss_top + loss_bottom

        _, preds = torch.max(output, dim=1)

        correct = float((label.int() == preds.int()).sum())
        accuracy = correct / len(label)

        optimizer.zero_grad()
        loss.backward()
        
        Clip_Norm = 1
        nn.utils.clip_grad_norm_(
            model['resnet'].parameters(), Clip_Norm, norm_type=2)
        nn.utils.clip_grad_norm_(
            model['fc_top'].parameters(), Clip_Norm, norm_type=2)
        nn.utils.clip_grad_norm_(
            model['fc_bottom'].parameters(), Clip_Norm, norm_type=2)
        nn.utils.clip_grad_norm_(
            model['classifier'].parameters(), Clip_Norm, norm_type=2)
        nn.utils.clip_grad_norm_(
            model['classifier_top'].parameters(), Clip_Norm, norm_type=2)
        nn.utils.clip_grad_norm_(
            model['classifier_bottom'].parameters(), Clip_Norm, norm_type=2)

        optimizer.step()

        if i % print_freq == 0:
            print('Train:\t'
                  'Epoch:[{0}][{1}/{2}]   \t'
                  'Acc: {acc:.3f}\t'
                  'Label_Loss: {l_loss:.4f}\t'
                  'Top_Loss: {t_loss:.4f}\t'
                  'Bottom_Loss: {b_loss:.4f}\t'
                  'Loss: {loss:.4f}\t'.format(
                      epoch, i + 1, len(train_dataloader), acc=accuracy,
                      l_loss=loss_label, t_loss=loss_top, b_loss=loss_bottom, loss=loss))
            
            
def validate(validate_dataloader, model, criterion, epoch):

    model['resnet'].eval()
    model['fc_top'].eval()
    model['fc_bottom'].eval()
    model['classifier'].eval()
    model['classifier_top'].eval()
    model['classifier_bottom'].eval()

    losses = 0
    correctes = 0
    preds_return = torch.LongTensor([])
    target_return = torch.LongTensor([])

    sample_file = {}

    with torch.no_grad():
        for sample in validate_dataloader:
            input, target = sample['image'].to(device), sample['label'].to(device)

            output_resnet_top, output_resnet_bottom = model['resnet'](input)
            output_fc_top = model['fc_top'](output_resnet_top)
            output_fc_bottom = model['fc_bottom'](output_resnet_bottom)

            output_model = torch.cat((output_fc_top, output_fc_bottom), 1)

            output = model['classifier'](output_model)

            loss = criterion(output, target)

            _, preds = torch.max(output, dim=1)

            preds_return = torch.cat((preds_return, preds.cpu()), 0)
            target_return = torch.cat((target_return, target.cpu()), 0)

            losses += loss

    return preds_return, target_return, losses


In [20]:
def LOSO(features, df, epochs=200, lr=0.01, batch_size=128, dropout=0.5, weight_decay=0.001,
         verbose=True):
    outputs_list = []
    #groupby reorders elements, now the labels are in same order as outputs
    df_groupby = pd.concat([i[1] for i in df.groupby("subject")])
    dataset_groupby = df_groupby["dataset"]
    
    le = LabelEncoder()
    labels = le.fit_transform(df["emotion"])
    labels_groupby = le.transform(df_groupby["emotion"])

    #loop over each subject
    for group in df.groupby("subject"):
        subject = group[0]
        #split data to train and test based on the subject index
        train_index = df[df["subject"] != subject].index
        X_train = features[train_index, :]
        y_train = labels[train_index]
        
        test_index = df[df["subject"] == subject].index
        X_test = features[test_index, :]
        y_test = labels[test_index]

        #create pytorch dataloaders from the split
        megc_dataset_train = MEData(X_train, y_train, transform)
        train_loader = torch.utils.data.DataLoader(megc_dataset_train,
                                                             batch_size=batch_size, shuffle=True,
                                                             num_workers=0)

        megc_dataset_test = MEData(X_test, y_test, transform)
        test_loader = torch.utils.data.DataLoader(megc_dataset_test,
                                                         batch_size=100, shuffle=False,
                                                         num_workers=0)

        
        model = Flow_Part_npic()
        model['resnet'] = model['resnet'].to(device)
        model['fc_top'] = model['fc_top'].to(device)
        model['fc_bottom'] = model['fc_bottom'].to(device)
        model['fc_top'] = model['fc_top'].apply(weight_init)
        model['fc_bottom'] = model['fc_bottom'].apply(weight_init)

        model['classifier'] = model['classifier'].to(device)
        model['classifier_top'] = model['classifier_top'].to(device)
        model['classifier_bottom'] = model['classifier_bottom'].to(device)

        model['classifier'] = model['classifier'].apply(weight_init)
        model['classifier_top'] = model['classifier_top'].apply(weight_init)
        model['classifier_bottom'] = model['classifier_bottom'].apply(
            weight_init)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam([
            {'params': model['resnet'].parameters(), 'lr':0.00001},
            {'params': model['fc_top'].parameters(), 'lr':LEARNING_RATE},
            {'params': model['fc_bottom'].parameters(), 'lr':LEARNING_RATE},
            {'params': model['classifier'].parameters(), 'lr':LEARNING_RATE},
            {'params': model['classifier_top'].parameters(),
             'lr':LEARNING_RATE},
            {'params': model['classifier_bottom'].parameters(),
             'lr':LEARNING_RATE},
        ], weight_decay=weight_decay)
        
        
        Epoch_F1_score = 0
        for epoch in range(epochs):
            if epoch % 10 == 0 and epoch != 0:
                if optimizer.param_groups[2]['lr'] > 0.00001:
                    optimizer.param_groups[1]['lr'] = optimizer.param_groups[1]['lr'] * 0.5
                    optimizer.param_groups[2]['lr'] = optimizer.param_groups[2]['lr'] * 0.5
                    optimizer.param_groups[3]['lr'] = optimizer.param_groups[3]['lr'] * 0.5
                    optimizer.param_groups[4]['lr'] = optimizer.param_groups[4]['lr'] * 0.5
                    optimizer.param_groups[5]['lr'] = optimizer.param_groups[5]['lr'] * 0.5
                    
            train(train_loader, model, criterion, optimizer, epoch)
            preds, target, loss = validate(test_loader, model, criterion, epoch)
            
            preds = preds.cpu().detach().data.numpy()
            test_f1 = f1_score(preds, target.cpu().data.numpy(), average="macro")
            
            
            if test_f1 >= Epoch_F1_score:
                Epoch_F1_score = test_f1
                best_preds = preds
            #Print statistics
            if verbose:
                print("Subject: {}, n={} | test_f1: {:.5} |best_f1: {:.5}".format(
                    subject, str(target.shape[0]).zfill(2), test_f1, Epoch_F1_score))
        outputs_list.append(preds)
            
    outputs = np.concatenate(outputs_list)
    f1_total = f1_score(labels_groupby, outputs, average="macro")
    idx = dataset_groupby == "smic"
    f1_smic = f1_score(labels_groupby[idx], outputs[idx], average="macro")
    idx = dataset_groupby == "casme2"
    f1_casme2 = f1_score(labels_groupby[idx], outputs[idx], average="macro")
    idx = dataset_groupby == "samm"
    f1_samm = f1_score(labels_groupby[idx], outputs[idx], average="macro")
    print("Total f1: {}, SMIC: {}, CASME2: {}, SAMM: {}".format(f1_total, f1_smic, f1_casme2, f1_samm))